In [2]:
%load_ext sql
%sql sqlite:///chinook.db

# Часть 1

## Задание 1

Вывести 20 самых больших по размеру треков жанра ROCK и формата MPEG (используйте команду LIMIT)

In [37]:
%%sql select t.name, t.bytes,g.name as Genre, m.name as Type from tracks t
LEFT JOIN genres g ON t.Genreid = g.genreid 
LEFT JOIN media_types m on t.mediatypeid = m.mediatypeid
WHERE g.name regexp "Rock" and m.name regexp "MPEG"
ORDER BY t.bytes desc
LIMIT 20;

 * sqlite:///chinook.db
Done.


Name,Bytes,Genre,Type
Dazed And Confused,52490554,Rock,MPEG audio file
Space Truckin',39267613,Rock,MPEG audio file
Dazed And Confused,36052247,Rock,MPEG audio file
We've Got To Get Together/Jingo,34618222,Rock,MPEG audio file
Funky Piano,30200730,Rock,MPEG audio file
Going Down / Highway Star,29846063,Rock,MPEG audio file
Santana Jam,29207100,Rock,MPEG audio file
The Sun Road,29008407,Rock,MPEG audio file
Whole Lotta Love,28191437,Rock,MPEG audio file
Mistreated (Alternate Version),27775442,Rock,MPEG audio file


## Задание 2
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра ROCK, приобретенные сотрудниками Microsoft

In [25]:
%%sql select c.company as Company, it.trackid as ID, t.name as Title, r.name as Artist, a.title as Album from customers c
left join invoices i on c.customerid = i.customerid
left join invoice_items it on it.invoiceid = i.invoiceid
left join tracks t on t.trackid = it.trackid
left join albums a on t.albumid = a.albumid
left join artists r on r.artistid = a.artistid
where (c.company like "Microsoft%") and (t.genreid = 1);

 * sqlite:///chinook.db
Done.


Company,ID,Title,Artist,Album
Microsoft Corporation,1166,Double Talkin' Jive,Guns N' Roses,Use Your Illusion I
Microsoft Corporation,1168,The Garden,Guns N' Roses,Use Your Illusion I
Microsoft Corporation,1170,Don't Damn Me,Guns N' Roses,Use Your Illusion I
Microsoft Corporation,1172,Dead Horse,Guns N' Roses,Use Your Illusion I
Microsoft Corporation,990,No Way Back,Foo Fighters,In Your Honor [Disc 1]
Microsoft Corporation,999,Still,Foo Fighters,In Your Honor [Disc 2]
Microsoft Corporation,1008,Razor,Foo Fighters,In Your Honor [Disc 2]
Microsoft Corporation,1026,My Hero,Foo Fighters,The Colour And The Shape


## Задание 3
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и общее количество, причем вывести только те наборы, для которых все треки стоят больше 1,5$ и для которых в наборе есть хотя бы один трек


In [93]:
%%sql
with sset AS (
    select distinct g.name as Genre, m.name as Type, t.trackid from tracks t
    left join genres g on t.genreid = g.genreid
    left join media_types m on t.mediatypeid = m.mediatypeid
)
select s.Genre,s.Type, avg(it.unitprice) from invoice_items it
inner join sset s using(Trackid)


limit 40;

 * sqlite:///chinook.db
(sqlite3.OperationalError) misuse of aggregate: count()
[SQL: with sset AS (
    select distinct g.name as Genre, m.name as Type, t.trackid from tracks t
    left join genres g on t.genreid = g.genreid
    left join media_types m on t.mediatypeid = m.mediatypeid
)
select s.Genre,s.Type, avg(it.unitprice) from invoice_items it
inner join sset s using(Trackid)
where count(it.trackid)>1

limit 40;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Задание 4
Для каждой компании вывести общее число заказов их клиентов. Необходимо исключить клиентов без компании (NULL)

In [22]:
%%sql select c.Company as Company, count(i.Invoiceid) from customers c 
left join invoices i on i.customerid  = c.customerid
where c.Company is not null
group by Company
limit 20;

 * sqlite:///chinook.db
Done.


Company,count(i.Invoiceid)
Apple Inc.,7
Banco do Brasil S.A.,7
Embraer - Empresa Brasileira de Aeronáutica S.A.,7
Google Inc.,7
JetBrains s.r.o.,7
Microsoft Corporation,7
Riotur,7
Rogers Canada,7
Telus,7
Woodstock Discos,7


## Задание 5
Для каждой компании вывести общее количестов песен, купленных по жанру поп-музыки

In [ ]:
%%sql select * from customers c
left join invoices i on c.customerid = i.customerid

limit 10;

## Задание 6
Для каждого артиста определить все жанры песен, которые у него есть и количество песен по каждому жанру

In [79]:
%%sql select
r.name as Artist,
g.name as Genre,
count(t.trackid) as Tracks
from genres g
left join tracks t on g.genreid = t.genreid
left join albums a on a.albumid = t.albumid
left join artists r on r.artistid = a.artistid
group by Artist
limit 25;

 * sqlite:///chinook.db
Done.


Artist,Genre,Tracks
AC/DC,Rock,18
Aaron Copland & London Symphony Orchestra,Classical,1
Aaron Goldberg,Jazz,1
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,2
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,Classical,1
"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair",Classical,1
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,1
Accept,Rock,4
Adrian Leaper & Doreen de Feis,Classical,1
Aerosmith,Rock,15


## Задание 7
Вывести средний размер альбома в байтах

In [41]:
%%sql select sum(t.bytes)/count(t.albumid) from tracks t
limit 10;

 * sqlite:///chinook.db
Done.


sum(t.bytes)/count(t.albumid)
33510207


# Часть 2

## Задание 8
Вывести названия треков и их продолжительность, которые были приобретены компаниями, которые обслуживаются сотрудниками, нанятыми после 3 сентября 2002 года

## Задание 9
Для каждого артиста найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение общей суммы, заработанной артистом по всем другим жанрам к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, и оно является максимальным для артиста, то выбирается жанр раньше по алфавиту

## Задание 10
Вывести альбом, который является минимальным по объему (в байтах) среди альбомов со стоимостью, большей средней цены за альбом по всем альбомам с таким же жанром (если у альбома песни с нескольким жанрами, то жанром является тот, у которого наибольшее количество песен в альбоме)